In [23]:
import pandas as pd
import re
# filename = './56878_v1-full_wc.tsv'
# filename = './webapp/samples/57385_v1-full.tsv'
# filename = 'webapp/samples/57474_v1-full.tsv'
filename = 'webapp/samples/57549_v1-full.tsv'
df = pd.read_csv(filename, delimiter='\t', header=2)

df.insert(allow_duplicates=True, loc = 13, value = 0, column = 'allele_frequency_%_corrected')
df.insert(allow_duplicates=True, loc = 14, value = 0, column = 'zygosity')
df.insert(allow_duplicates=True, loc = 15, value = 1, column = 'entry_count')

# delete entries with multiple allele_frequencies
corr_freq = df.loc[:, ['allele_frequency_%']]

drop_index = []
for index,value in corr_freq['allele_frequency_%'].items():
    val = re.search('[ATGCatgc=]', value)
    if val is not None:
        drop_index.append(index)

df.drop(drop_index, inplace = True)
corr_freq.drop(drop_index, inplace = True)

# populate corrected freq column
corr_freq = corr_freq.astype('float')

corr_freq = corr_freq.apply(lambda x: 100 - x)

df['allele_frequency_%_corrected'] = corr_freq

df.iloc[:, [1,2,3,10,11,12,13,14,15]].head()


,type,ref,length,allele_coverage,allele_ratio,allele_frequency_%,allele_frequency_%_corrected,zygosity,entry_count
0,SNV,C,1,"C=211,T=189","C=0.5275,T=0.4725",47.25,52.75,0,1
1,SNV,G,1,"G=222,C=178","G=0.555,C=0.445",44.50,55.50,0,1
2,SNV,A,1,"A=0,G=372","A=0.0,G=1.0",100.00,0.00,0,1
3,SNV,C,1,"C=181,T=218","C=0.4536,T=0.5464",54.64,45.36,0,1
4,SNV,G,1,"G=0,A=386","G=0.0,A=1.0",100.00,0.00,0,1


In [24]:
df_snp = df[df['type'] == 'SNV']
homozygous = df_snp['allele_frequency_%_corrected'].map(lambda x: 'homozygous' if x < 10 else 'heterozygous' )
df_snp['zygosity'] = homozygous
df.update(df_snp)

/var/folders/pt/vcfrx50j7vzg3gkz0bt7ykdr0000gn/T/ipykernel_25878/1153009908.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_snp['zygosity'] = homozygous


In [25]:
cols = [1,2,10,11,12,13,14]
df.iloc[170::, cols]

,type,ref,allele_coverage,allele_ratio,allele_frequency_%,allele_frequency_%_corrected,zygosity


In [26]:
df.iloc[1:5,cols]

,type,ref,allele_coverage,allele_ratio,allele_frequency_%,allele_frequency_%_corrected,zygosity
1,SNV,G,"G=222,C=178","G=0.555,C=0.445",44.50,55.50,heterozygous
2,SNV,A,"A=0,G=372","A=0.0,G=1.0",100.00,0.00,homozygous
3,SNV,C,"C=181,T=218","C=0.4536,T=0.5464",54.64,45.36,heterozygous
4,SNV,G,"G=0,A=386","G=0.0,A=1.0",100.00,0.00,homozygous


In [27]:

df_indel = df[df['type'] == 'INDEL']


In [28]:
zygosity = df_indel['allele_frequency_%_corrected'].map(lambda x: 'homozygous' if x < 20 else 'heterozygous')
print(zygosity)
df_indel['zygosity'] = zygosity
df.update(df_indel)
df.iloc[[9,15,42,151],cols]

9        homozygous
14     heterozygous
36     heterozygous
49       homozygous
72       homozygous
73     heterozygous
91       homozygous
101      homozygous
113    heterozygous
116    heterozygous
126      homozygous
129    heterozygous
130    heterozygous
140    heterozygous
144    heterozygous
Name: allele_frequency_%_corrected, dtype: object


/var/folders/pt/vcfrx50j7vzg3gkz0bt7ykdr0000gn/T/ipykernel_25878/1824082228.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_indel['zygosity'] = zygosity


,type,ref,allele_coverage,allele_ratio,allele_frequency_%,allele_frequency_%_corrected,zygosity
9,INDEL,GT,"GT=0,G=396","GT=0.0,G=1.0",100.00,0.00,homozygous
16,SNV,G,"G=186,T=213","G=0.4662,T=0.5338",53.38,46.62,heterozygous
43,SNV,G,"G=206,C=194","G=0.515,C=0.485",48.50,51.50,heterozygous
156,SNV,A,"A=0,G=399","A=0.0,G=1.0",100.00,0.00,homozygous


In [29]:
df.to_csv(f'./processed_{filename.split("/")[-1]}', sep='\t', index = False)